In [44]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json 

In [45]:
def load_data(currency):
    folder = f"data"
    all_data = []

    if not os.path.exists(folder):
        return pd.DataFrame()
    
    for file in sorted(os.listdir(folder)):
        if file.endswith(".json"):
            with open(os.path.join(folder, file), "r") as f:
                data = json.load(f)
                print(json.dumps(data, indent=4))
                try:
                    items = data["exchangeRates"]["item"]
                    date = data["exchangeRates"]["@currencyDate"]
                    if isinstance(items, dict):
                        items = [items]
                    for rate in items:
                        to_code = rate["@currencyCode"]
                        value = float(rate["inverseRate"])
                        all_data.append({"date": date, "to": to_code, "rate": value})
                except Exception as e:
                    pass
    df = pd.DataFrame(all_data)
    if not df.empty:
        df["date"] = pd.to_datetime(df["date"])
    return df

In [46]:
currencies = ["USD", "EUR", "GBP", "JPY", "CNY"]
all_frames = {}

for c in currencies:
    all_frames[c] = load_data(c)

In [42]:
for currency, df in all_frames.items():
    if df.empty:
        print(f"No data for {currency}")
        continue

    top_currencies = df["to"].value_counts().head(3).index

    for to_curr in top_currencies:
        sub = df[df["to"] == to_curr]

        plt.figure(figsize=(10, 4))
        sns.lineplot(data=sub, x="date", y="rate")
        plt.title(f"{currency} to {to_curr} exchange rate over time")
        plt.xlabel("Date")
        plt.ylabel("Exchange Rate")
        plt.show()

No data for USD
No data for EUR
No data for GBP
No data for JPY
No data for CNY


In [43]:
for currency, df in all_frames.items():
    if df.empty:
        continue

    df = df.sort_values(["to", "date"])
    df["change"] = df.groupby("to")["rate"].diff().abs()

    mean_volatility = df.groupby("to")["change"].mean().sort_values(ascending=False)
    print(f"Volatility for {currency}:")
    print(mean_volatility.head(3))
    print()

###Observations (Quest Book)

1. Some currencies like JPY and CNY show smoother trends.
2. Volatility increased around early 2020, possibly due to COVID-19.
3. USD and EUR are among the most stable base currencies.
4. Major economic events like Brexit in 2016 may show sharp spikes in GBP-related pairs.
5. Exchange rates fluctuate more when base is from developing countries. 